In [2]:
import json
import numpy as np
import pandas as pd
import requests
import numpy
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt

# Research Question: Find the top 10 players with the highest ratio between completed passes and attempted passes for La Liga

### Import the players dataset

In [17]:

players=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\players.json")
players
#Displaying maximum number
pd.set_option('display.max_columns', 14)
players.head()

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konat\u00e9,null


### Import the event data for France

In [4]:
events_fra=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\events\\events_France.json")
pd.set_option('display.max_columns', 15)
events_fra.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],253784,"[{'y': 51, 'x': 50}, {'y': 46, 'x': 31}]",2500686,Pass,3799,1H,1.935181,85,176505119
1,8,High pass,[{'id': 1801}],29474,"[{'y': 46, 'x': 31}, {'y': 74, 'x': 68}]",2500686,Pass,3799,1H,3.599295,83,176505121
2,1,Air duel,"[{'id': 703}, {'id': 1801}]",253784,"[{'y': 74, 'x': 68}, {'y': 54, 'x': 72}]",2500686,Duel,3799,1H,6.827043,10,176505122
3,1,Air duel,"[{'id': 701}, {'id': 1802}]",56441,"[{'y': 26, 'x': 32}, {'y': 46, 'x': 28}]",2500686,Duel,3772,1H,6.985577,10,176505017
4,1,Ground attacking duel,"[{'id': 702}, {'id': 1801}]",366760,"[{'y': 54, 'x': 72}, {'y': 55, 'x': 73}]",2500686,Duel,3799,1H,9.511272,11,176505124


#### As we have to calculate the efficiency of the players based on the passes, we filter out the information related to passes. Thus we target the column 'eventName' and choose only those rows which contains information related to passes. 

In [5]:
events_fra['eventName']=events_fra['eventName'].astype(str)
events_fra=events_fra[(events_fra.eventName.str.contains('Pass'))]
pd.set_option('display.max_columns', 15)
events_fra.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],253784,"[{'y': 51, 'x': 50}, {'y': 46, 'x': 31}]",2500686,Pass,3799,1H,1.935181,85,176505119
1,8,High pass,[{'id': 1801}],29474,"[{'y': 46, 'x': 31}, {'y': 74, 'x': 68}]",2500686,Pass,3799,1H,3.599295,83,176505121
10,8,Simple pass,[{'id': 1801}],56441,"[{'y': 40, 'x': 27}, {'y': 40, 'x': 34}]",2500686,Pass,3772,1H,13.710413,85,176505021
12,8,Hand pass,[{'id': 1801}],26245,"[{'y': 46, 'x': 13}, {'y': 86, 'x': 34}]",2500686,Pass,3799,1H,19.984771,81,176505135
13,8,Simple pass,[{'id': 1801}],26086,"[{'y': 86, 'x': 34}, {'y': 39, 'x': 32}]",2500686,Pass,3799,1H,26.513924,85,176505137


#### Splitting the column 'Tags' that is a dictionary

In [6]:
TAG1=[]
for i in range(len(events_fra.tags)):
    try:
        TAG1.append(str(events_fra.tags[i][0]['id']))
    except:
        TAG1.append('Notag')
TAG2=[]
for i in range(len(events_fra.tags)):
    try:
        TAG2.append(str(events_fra.tags[i][1]['id']))
    except:
        TAG2.append('Notag')
TAG3=[]
for i in range(len(events_fra.tags)):
    try:
        TAG3.append(str(events_fra.tags[i][2]['id']))
    except:
        TAG3.append('Notag')
TAG4=[]
for i in range(len(events_fra.tags)):
    try:
        TAG4.append(str(events_fra.tags[i][2]['id']))
    except:
        TAG4.append('Notag')
TAG1=pd.DataFrame({'Tag1':TAG1})
TAG2=pd.DataFrame({'Tag2':TAG2})
TAG3=pd.DataFrame({'Tag3':TAG3})
TAG4=pd.DataFrame({'Tag4':TAG4})
TAG=[TAG1,TAG2,TAG3,TAG4]
TAG=pd.concat(TAG ,axis=1, sort=False)
mix_events_fra=[events_fra,TAG]
events_fra=pd.concat(mix_events_fra,  axis=1,  sort=False).reindex(events_fra.index)
events_fra=events_fra.drop('tags',axis=1)

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,253784.0,"[{'y': 51, 'x': 50}, {'y': 46, 'x': 31}]",2500686.0,Pass,3799.0,1H,1.935181,85,176505119.0,1801,Notag,Notag,Notag
1,8.0,High pass,29474.0,"[{'y': 46, 'x': 31}, {'y': 74, 'x': 68}]",2500686.0,Pass,3799.0,1H,3.599295,83,176505121.0,1801,Notag,Notag,Notag
10,8.0,Simple pass,56441.0,"[{'y': 40, 'x': 27}, {'y': 40, 'x': 34}]",2500686.0,Pass,3772.0,1H,13.710413,85,176505021.0,1801,Notag,Notag,Notag
12,8.0,Hand pass,26245.0,"[{'y': 46, 'x': 13}, {'y': 86, 'x': 34}]",2500686.0,Pass,3799.0,1H,19.984771,81,176505135.0,1801,Notag,Notag,Notag
13,8.0,Simple pass,26086.0,"[{'y': 86, 'x': 34}, {'y': 39, 'x': 32}]",2500686.0,Pass,3799.0,1H,26.513924,85,176505137.0,1801,Notag,Notag,Notag


In [7]:
events_fra['eventName'] = events_fra['eventName'].astype(str)
events_fra=events_fra[(events_fra.eventName.str.contains('Pass'))]
events_fra.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,253784.0,"[{'y': 51, 'x': 50}, {'y': 46, 'x': 31}]",2500686.0,Pass,3799.0,1H,1.935181,85,176505119.0,1801,Notag,Notag,Notag
1,8.0,High pass,29474.0,"[{'y': 46, 'x': 31}, {'y': 74, 'x': 68}]",2500686.0,Pass,3799.0,1H,3.599295,83,176505121.0,1801,Notag,Notag,Notag
10,8.0,Simple pass,56441.0,"[{'y': 40, 'x': 27}, {'y': 40, 'x': 34}]",2500686.0,Pass,3772.0,1H,13.710413,85,176505021.0,1801,Notag,Notag,Notag
12,8.0,Hand pass,26245.0,"[{'y': 46, 'x': 13}, {'y': 86, 'x': 34}]",2500686.0,Pass,3799.0,1H,19.984771,81,176505135.0,1801,Notag,Notag,Notag
13,8.0,Simple pass,26086.0,"[{'y': 86, 'x': 34}, {'y': 39, 'x': 32}]",2500686.0,Pass,3799.0,1H,26.513924,85,176505137.0,1801,Notag,Notag,Notag


#### Counting the number of passes with respect to the player id

In [8]:
passes=events_fra
passes=passes.drop('positions',axis=1)
passes.playerId=list(passes.playerId)
PLAYERS=[]
for i in range(len(passes.playerId)):
    PLAYERS.append(passes.playerId.iloc[i])
passes_player_fra=pd.DataFrame(PLAYERS , columns=['PlayerId'])
passes_player_fra['Numberofpasses'] = passes_player_fra.groupby('PlayerId')['PlayerId'].transform('count')
passes_player_fra

,PlayerId,Numberofpasses
0,253784.0,64
1,29474.0,1130
2,56441.0,1497
3,26245.0,169
4,26086.0,1070
...,...,...
319193,263802.0,784
319194,3450.0,504
319195,295761.0,895
319196,249926.0,266


#### In order to set the threshold of the number of passes, we calculate the first quartile of the number of passes. Then we set the lower limit as mean-1.5$\times$sd. The reason behind this is, while dealing with a set of variables, we could give mean-2$\times$ sd. But the value of twice of sd exceeds the mean. So, we go for mean-1.5$\times$sd. Here we are concerned about the players who have played very limited number of matches or insufficient amount of hours. That is why we set the lower limit as the only threshold.

In [9]:
#calculating Mean
#Calculating Standard deviation
Mean=passes_player_fra.Numberofpasses.mean()
std=passes_player_fra.Numberofpasses.std()
Mean=int(Mean)
std=int(std)
#Calculating Lower bound
lower_bound=Mean-std
ID_passes_threshold_fra=[]
frequency_passes_threshold_fra=[]
frequency_passes=[]
ID_passes=[]
for i in range(len(passes_player_fra.Numberofpasses)):
    frequency_passes.append(float(passes_player_fra.Numberofpasses.iloc[i]))
    ID_passes.append(float(passes_player_fra.PlayerId.iloc[i]))
for i in range(len(frequency_passes)):
    if frequency_passes[i]>lower_bound:
        frequency_passes_threshold_fra.append(int(frequency_passes[i]))
        ID_passes_threshold_fra.append(int(ID_passes[i]))

In [10]:
passes_threshold_fra=pd.DataFrame(ID_passes_threshold_fra, columns=['PlayerId'])
passes_threshold_fra.insert(1, "Numberofpasses",frequency_passes_threshold_fra, True)
passes_threshold_fra.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
passes_threshold_fra

,PlayerId,Numberofpasses
0,29474,1130
1,56441,1497
2,26086,1070
5,26265,1269
6,274344,1138
...,...,...
136053,410062,827
140676,25649,1251
147125,41127,611
156078,14801,812


In [11]:
events_fra_tag2=events_fra.loc[events_fra['Tag2'] == '1801']
events_fra_tag1=events_fra.loc[events_fra['Tag1']=='1801']
events_fra_tag3=events_fra.loc[events_fra['Tag3']=='1801']
events_fra_tag4=events_fra.loc[events_fra['Tag4']=='1801']
events_fra_tag=[events_fra_tag1,events_fra_tag2,events_fra_tag3,events_fra_tag4]
success_fra=pd.concat(events_fra_tag, axis=0)
success_fra.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,253784.0,"[{'y': 51, 'x': 50}, {'y': 46, 'x': 31}]",2500686.0,Pass,3799.0,1H,1.935181,85,176505119.0,1801,Notag,Notag,Notag
1,8.0,High pass,29474.0,"[{'y': 46, 'x': 31}, {'y': 74, 'x': 68}]",2500686.0,Pass,3799.0,1H,3.599295,83,176505121.0,1801,Notag,Notag,Notag
10,8.0,Simple pass,56441.0,"[{'y': 40, 'x': 27}, {'y': 40, 'x': 34}]",2500686.0,Pass,3772.0,1H,13.710413,85,176505021.0,1801,Notag,Notag,Notag
12,8.0,Hand pass,26245.0,"[{'y': 46, 'x': 13}, {'y': 86, 'x': 34}]",2500686.0,Pass,3799.0,1H,19.984771,81,176505135.0,1801,Notag,Notag,Notag
13,8.0,Simple pass,26086.0,"[{'y': 86, 'x': 34}, {'y': 39, 'x': 32}]",2500686.0,Pass,3799.0,1H,26.513924,85,176505137.0,1801,Notag,Notag,Notag


In [12]:
S=[]
for i in range(len(success_fra.playerId)):
   S.append(success_fra.playerId.iloc[i])
success_fra_new=pd.DataFrame(S , columns=['PlayerId'])
success_fra_new['Numberofpasses'] = success_fra_new.groupby('PlayerId')['PlayerId'].transform('count')
success_fra_new.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
success_fra_new

,PlayerId,Numberofpasses
0,253784.0,47
1,29474.0,697
2,56441.0,682
3,26245.0,143
4,26086.0,469
...,...,...
120238,397030.0,16
121479,26131.0,6
121956,25503.0,20
122125,37796.0,10


In [15]:
final_passes_fra=pd.merge(success_fra_new, passes_threshold_fra, on='PlayerId')
final_passes_fra=final_passes_fra.rename(columns={'Numberofpasses_x':'AccuratePassesNumber','Numberofpasses_y':'TotalPassesNumber'})
final_passes_fra

,PlayerId,AccuratePassesNumber,TotalPassesNumber
0,29474.0,697,1130
1,56441.0,682,1497
2,26086.0,469,1070
3,26265.0,521,1269
4,274344.0,379,1138
...,...,...,...
244,7986.0,270,563
245,20677.0,139,603
246,301013.0,158,1228
247,25710.0,234,551


In [21]:
def make_col(df, col):
    return pd.concat([df.drop([col], axis=1), df[col].apply(pd.Series)], axis=1)
teams = pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\teams.json").rename(columns={'name':'teamname'})
teams = make_col(teams, "area")
teams = teams.drop(columns='city')
teams = teams.rename(columns={'name':'nation'})
frateams = teams[(teams["alpha3code"]=="FRA") & (teams["type"]=="club")]
fraplayer = players.merge(frateams, left_on="currentTeamId", right_on="wyId")
fraplayer=fraplayer.drop(columns=['birthArea','height','role','currentNationalTeamId','officialName','type','middleName','lastName','passportArea','weight','currentTeamId','firstName','foot'],axis=1)

fraplayer=fraplayer.drop(columns=['alpha3code','alpha2code','id','nation','wyId_y','teamname'])
fraplayer=fraplayer.rename(columns={"wyId_x": "PlayerId"})

final_player_passes_fra=pd.concat([fraplayer, final_passes_fra],axis=1)
final_player_passes_fra


,birthDate,PlayerId,shortName,PlayerId,AccuratePassesNumber,TotalPassesNumber
0,1999-01-23,393228,M. Sarr,29474.0,697.0,1130.0
1,1996-04-11,328488,P. Burner,56441.0,682.0,1497.0
2,1991-07-19,131959,J. Seri,26086.0,469.0,1070.0
3,1995-09-07,266286,Marlon,26265.0,521.0,1269.0
4,1997-09-22,301016,R. Perraud,274344.0,379.0,1138.0
...,...,...,...,...,...,...
477,1991-09-29,30427,J. Mari\u00e9,NaN,NaN,NaN
478,1988-11-28,29843,A. Bouka Moutou,NaN,NaN,NaN
479,1986-10-16,29748,F. Chafik,NaN,NaN,NaN
480,1996-08-19,393214,V. Rosier,NaN,NaN,NaN


In [22]:
Efficiency_fra=[]
for i in range(len(final_player_passes_fra.AccuratePassesNumber)):
    total=final_player_passes_fra.TotalPassesNumber.iloc[i]
    accurate=final_player_passes_fra.AccuratePassesNumber.iloc[i]
    Ratio=accurate/total
    Efficiency_fra.append(Ratio)
Efficiency_fra=pd.DataFrame(Efficiency_fra)

In [23]:
final_player_passes_fra.insert(4,'Efficiency',Efficiency_fra)
final_player_ratio_fra=final_player_passes_fra
final_player_ratio_fra

,birthDate,PlayerId,shortName,PlayerId,Efficiency,AccuratePassesNumber,TotalPassesNumber
0,1999-01-23,393228,M. Sarr,29474.0,0.616814,697.0,1130.0
1,1996-04-11,328488,P. Burner,56441.0,0.455578,682.0,1497.0
2,1991-07-19,131959,J. Seri,26086.0,0.438318,469.0,1070.0
3,1995-09-07,266286,Marlon,26265.0,0.410559,521.0,1269.0
4,1997-09-22,301016,R. Perraud,274344.0,0.333040,379.0,1138.0
...,...,...,...,...,...,...,...
477,1991-09-29,30427,J. Mari\u00e9,NaN,NaN,NaN,NaN
478,1988-11-28,29843,A. Bouka Moutou,NaN,NaN,NaN,NaN
479,1986-10-16,29748,F. Chafik,NaN,NaN,NaN,NaN
480,1996-08-19,393214,V. Rosier,NaN,NaN,NaN,NaN


In [25]:
final_player_ratio_ita.to_json(r'C:\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw2\final_player_ratio_ita.json')

In [24]:
bestefficiency_fra=final_player_ratio_fra.nlargest(10, ['Efficiency']) 
bestefficiency_fra=bestefficiency_fra.reset_index()
bestefficiency_fra=bestefficiency_fra.drop(columns=['index','PlayerId'])
bestefficiency_fra=bestefficiency_fra.set_value(9, 'shortName', 'V.Lindelof')

bestefficiency_fra

C:\Users\Debodeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


,birthDate,shortName,Efficiency,AccuratePassesNumber,TotalPassesNumber
0,1999-01-23,A. Lafont,0.933450,533.0,571.0
1,1990-08-12,M. Balotelli,0.900281,641.0,712.0
2,1995-06-15,H. Diallo,0.869504,613.0,705.0
3,1991-10-09,A. Delort,0.822828,483.0,587.0
4,1999-07-02,H. Mahou,0.815338,808.0,991.0
5,1991-07-10,Jonathan Cafu,0.783427,416.0,531.0
6,1992-08-16,M. Poundj\u00e9,0.777509,643.0,827.0
7,1997-12-16,J. Porsan-Clemente,0.757192,658.0,869.0
8,1995-03-05,M. Hassen,0.745554,545.0,731.0
9,1997-12-02,V.Lindelof,0.727586,422.0,580.0


### The above table shows the top 10 most efficient players. We can observe that the highest efficiency is 89.03% in the Serie-A.